## Define synapomorphies from alignment, for different taxonomic levels

In [3]:
import numpy as np
import pandas as pd
import sqlite3
#from __future__ import division
from Bio import AlignIO

### Start with synapomizer script had built before

In [4]:
# %load -r 22-32 /Users/hughcross/scripts/synapomizer.py
def number_alleles(column):
    leng = len(column)
    bp_list = []
    for bp in range(0,leng):
        pos = column[bp]
        if pos in bp_list:
            pass
        else:
            bp_list.append(pos)
    #no_alleles = len(bp_list) # maybe take out if a gap
    return bp_list

In [5]:
# %load -r 34-49 /Users/hughcross/scripts/synapomizer.py
def informative_alleles(character_dict):
    """a function to filter out uninformative positions"""
    new_dict = {}
    allele_list = []
    for k,v in character_dict.items():
        newd = v
        for key,value in newd.items():
            new_dict.setdefault(key, []).append(value)
    #print new_dict
    for keys, values in new_dict.items():
        alleles = set(values)
        #print keys
        #print alleles
        if len(alleles) > 1:
            allele_list.append(keys)
    return allele_list

In [6]:
# start with a file, then import muscle to make other alignments
# %load -r 9-12 /Users/hughcross/scripts/synapomizer.py
align_file = '/Users/hughcross/Analysis/CAMEL/cam2018/local_chenopod_ref_seqs alignment_for mapping.nex'
alignment = AlignIO.read(align_file, 'nexus')
length = alignment.get_alignment_length()

In [7]:
# %load -r 14-19 /Users/hughcross/scripts/synapomizer.py
#create a dictionary with sample# : genus name ## note: need to make this part flexible so that different taxonomic levels can be used
name_map = {} # also maybe make a list of each genus, and a dict where key = genus, values = number of species in genus
genus_dict = {}
sample_map = {}
for i, record in enumerate(alignment):
    fullname = record.id
    newname = fullname.split('_')[0]
    name_map[i]=newname
    genus_dict.setdefault(newname, []).append(fullname)
    sample_map[i]=fullname

In [9]:
sample_map[0]

'Salsola_kali_AD_b36'

In [10]:
genus_dict['Sclerolaena']

['Sclerolaena_cuneata_AD_b32', 'Sclerolaena_deserticola_AD_b31']

In [11]:
# make a dictionary of lists for each clade
bluebush = ['Maireana','Enchylaena','Neobassia','Sclerolaena','Threlkeldia']
saltbush =['Atriplex','Chenopodium','Rhagodia','Einadia','Dysphania']
genera = genus_dict.keys()
print(genera)

dict_keys(['Salsola', 'Tecticornia', 'Atriplex', 'Chenopodium', 'Rhagodia', 'Dysphania', 'Einadia', 'Bassia', 'Suaeda', 'Enchylaena', 'Maireana', 'Neobassia', 'Sclerolaena', 'Threlkeldia'])


In [12]:
clades = {}
for gen in genera:
    if gen in bluebush:
        clades[gen]='bluebush'
    elif gen in saltbush:
        clades[gen]='saltbush'
    else:
        clades[gen]=gen
print(clades)

{'Salsola': 'Salsola', 'Tecticornia': 'Tecticornia', 'Atriplex': 'saltbush', 'Chenopodium': 'saltbush', 'Rhagodia': 'saltbush', 'Dysphania': 'saltbush', 'Einadia': 'saltbush', 'Bassia': 'Bassia', 'Suaeda': 'Suaeda', 'Enchylaena': 'bluebush', 'Maireana': 'bluebush', 'Neobassia': 'bluebush', 'Sclerolaena': 'bluebush', 'Threlkeldia': 'bluebush'}


In [13]:
# %load -r 51-66 /Users/hughcross/scripts/synapomizer.py
pos_list = []
for num in range(0,length):
    position = alignment[:,num] # add num to list
    #print position
    allele_list = number_alleles(position)
    no_alleles = len(allele_list)
    
    if '-' in allele_list:
        no_alleles = no_alleles - 1
    #print no_alleles

    if no_alleles > 1:
        #print position
        pos_list.append(num)
print(pos_list)
print(len(pos_list))

[27, 35, 68, 70, 71, 77, 81, 97, 103, 110, 111, 114, 118, 119, 120, 121, 122, 127, 133, 136, 139]
21


In [14]:
print(length)

149


In [15]:
ambigs = {'AT':'W','TA':'W','CG':'S','GC':'S','CT':'Y','TC':'Y','AG':'R','GA':'R','AC':'M','CA':'M','GT':'K','TG':'K'}

In [16]:
# %load -r 68-85 /Users/hughcross/scripts/synapomizer.py
## note: fixing ambiguous characters in original below (was 'continue')
final_chars = {}

for pos in pos_list:
    gen_char_dict = {}
    for sample in range(0,32): # change this to num of samples
        char = alignment[sample,pos]
        genus = name_map[sample]
        
        gen_char_dict.setdefault(genus, []).append(char)
        
    for key, values in gen_char_dict.items():
        #print key
        newlist = gen_char_dict[key]
        #print newlist
        check = set(newlist)
        #print check
        size = len(check)
        if size > 2:
            #final_chars.setdefault(key, {})[pos]='N'
            newchar = 'N'
        elif size == 2:
            amb_str = ''
            for bp in check:
                amb_str = amb_str + bp
            if '-' in amb_str:
                newchar = amb_str.replace('-','') # may want to count the gaps and adjust later
            else:
                newchar = ambigs[amb_str]
                #final_chars.setdefault(key, {})[pos]=newchar
            
        else:
            newchar = newlist[0]
        final_chars.setdefault(key, {})[pos]=newchar
        

In [24]:
# %load -r 68-85 /Users/hughcross/scripts/synapomizer.py
## note: fixing ambiguous characters in original below (was 'continue')
final_chars_clade = {}

for pos in pos_list:
    cld_char_dict = {}
    for sample in range(0,32): # change this to num of samples
        char = alignment[sample,pos]
        genus = name_map[sample]
        clade = clades[genus]
        
        cld_char_dict.setdefault(clade, []).append(char)
        
    for key, values in cld_char_dict.items():
        #print key
        newlist = cld_char_dict[key]
        #print newlist
        check = set(newlist)
        #print check
        size = len(check)
        if size > 2:
            #final_chars.setdefault(key, {})[pos]='N'
            newchar = 'N'
        elif size == 2:
            amb_str = ''
            for bp in check:
                amb_str = amb_str + bp
            if '-' in amb_str:
                newchar = amb_str.replace('-','') # may want to count the gaps and adjust later
            else:
                newchar = ambigs[amb_str]
                #final_chars.setdefault(key, {})[pos]=newchar
            
        else:
            newchar = newlist[0]
        final_chars_clade.setdefault(key, {})[pos]=newchar
        

In [26]:
final_chars_clade['Bassia']

{27: 'C',
 35: 'T',
 68: 'T',
 70: 'A',
 71: 'T',
 77: 'T',
 81: 'T',
 97: 'C',
 103: 'T',
 110: 'G',
 111: 'R',
 114: 'A',
 118: 'G',
 119: 'R',
 120: 'A',
 121: 'W',
 122: 'A',
 127: '-',
 133: '-',
 136: '-',
 139: '-'}

In [27]:
# %load -r 68-85 /Users/hughcross/scripts/synapomizer.py
## note: fixing ambiguous characters in original below (was 'continue')

final_char_indiv = {}
for pos in pos_list:
    sample_char_dict = {}
    for sample in range(0,32): # change this to num of samples
        char = alignment[sample,pos]
        
        samp_indiv = sample_map[sample]
        
        sample_char_dict.setdefault(samp_indiv, []).append(char)
    for key, values in sample_char_dict.items():
        #print key
        newlist = sample_char_dict[key]
        #print newlist
        check = set(newlist)
        #print check
        size = len(check)
        if size > 2:
            #final_chars.setdefault(key, {})[pos]='N'
            newchar = 'N'
        elif size == 2:
            amb_str = ''
            for bp in check:
                amb_str = amb_str + bp
            if '-' in amb_str:
                newchar = amb_str.replace('-','') # may want to count the gaps and adjust later
            else:
                newchar = ambigs[amb_str]
                #final_chars.setdefault(key, {})[pos]=newchar
            
        else:
            newchar = newlist[0]
        
        final_char_indiv.setdefault(key, {})[pos]=newchar

In [28]:
final_char_indiv['Atriplex_deserticola_gi440584215']

{27: 'C',
 35: 'C',
 68: 'T',
 70: 'A',
 71: 'T',
 77: 'T',
 81: 'C',
 97: 'C',
 103: 'G',
 110: 'G',
 111: 'C',
 114: 'A',
 118: 'G',
 119: 'A',
 120: 'A',
 121: 'A',
 122: 'G',
 127: 'A',
 133: 'C',
 136: 'G',
 139: 'T'}

In [29]:
# %load -r 89-95 /Users/hughcross/scripts/synapomizer.py
# To filter out the characters that have only one alleles; it is variable but only one synapomorphy
## perhaps compare dictionaries in final_chars, if all at one position the same, then delete. 
### try to make it a function 

inform_alleles = informative_alleles(final_chars)
print(inform_alleles)
print(len(inform_alleles))

[27, 35, 68, 70, 71, 77, 81, 97, 103, 110, 111, 114, 118, 119, 120, 121, 122, 127, 133, 136, 139]
21


In [30]:
# %load -r 97-109 /Users/hughcross/scripts/synapomizer.py
# the informative_alleles function makes a list of informative alleles, then the next loop creates a filtered dictionary of dicts for each genus
filtered_chars1 = {}
for k,v in final_chars.items():
    tax_dict = v  #final_chars[k]
    #print tax_dict
    for key, value in tax_dict.items():
        if key in inform_alleles:
            filtered_chars1.setdefault(k, {})[key]=value

# now maybe sort the dictionary
print(filtered_chars1)
print(name_map)
#now to deal with gaps?, maybe later

{'Salsola': {27: 'C', 35: 'T', 68: 'T', 70: 'C', 71: 'T', 77: 'A', 81: 'C', 97: 'C', 103: 'T', 110: 'G', 111: 'G', 114: 'A', 118: 'G', 119: 'A', 120: 'A', 121: 'G', 122: 'A', 127: '-', 133: '-', 136: '-', 139: '-'}, 'Tecticornia': {27: 'C', 35: 'T', 68: 'T', 70: 'A', 71: 'T', 77: 'A', 81: 'C', 97: 'C', 103: 'T', 110: 'T', 111: 'A', 114: 'A', 118: 'G', 119: 'G', 120: 'A', 121: 'A', 122: '-', 127: '-', 133: '-', 136: '-', 139: '-'}, 'Atriplex': {27: 'C', 35: 'Y', 68: 'T', 70: 'A', 71: 'T', 77: 'T', 81: 'C', 97: 'C', 103: 'G', 110: 'G', 111: 'C', 114: 'A', 118: 'G', 119: 'A', 120: 'A', 121: 'A', 122: 'G', 127: 'A', 133: 'C', 136: 'G', 139: 'T'}, 'Chenopodium': {27: 'C', 35: 'T', 68: 'T', 70: 'M', 71: 'N', 77: 'Y', 81: 'Y', 97: 'C', 103: 'K', 110: 'G', 111: 'C', 114: 'R', 118: 'K', 119: 'A', 120: 'M', 121: 'N', 122: 'N', 127: 'N', 133: 'N', 136: 'N', 139: 'T'}, 'Rhagodia': {27: 'C', 35: 'T', 68: 'T', 70: 'A', 71: 'T', 77: 'T', 81: 'C', 97: 'C', 103: 'G', 110: 'G', 111: 'C', 114: 'A', 118: 

In [31]:
# %load -r 97-109 /Users/hughcross/scripts/synapomizer.py
# the informative_alleles function makes a list of informative alleles, then the next loop creates a filtered dictionary of dicts for each genus
filtered_chars_cld = {}
for k,v in final_chars_clade.items():
    tax_dict = v  #final_chars[k]
    #print tax_dict
    for key, value in tax_dict.items():
        if key in inform_alleles:
            filtered_chars_cld.setdefault(k, {})[key]=value

# now maybe sort the dictionary
print(filtered_chars_cld)
#print(name_map)
#now to deal with gaps?, maybe later

{'Salsola': {27: 'C', 35: 'T', 68: 'T', 70: 'C', 71: 'T', 77: 'A', 81: 'C', 97: 'C', 103: 'T', 110: 'G', 111: 'G', 114: 'A', 118: 'G', 119: 'A', 120: 'A', 121: 'G', 122: 'A', 127: '-', 133: '-', 136: '-', 139: '-'}, 'Tecticornia': {27: 'C', 35: 'T', 68: 'T', 70: 'A', 71: 'T', 77: 'A', 81: 'C', 97: 'C', 103: 'T', 110: 'T', 111: 'A', 114: 'A', 118: 'G', 119: 'G', 120: 'A', 121: 'A', 122: '-', 127: '-', 133: '-', 136: '-', 139: '-'}, 'saltbush': {27: 'C', 35: 'Y', 68: 'T', 70: 'M', 71: 'N', 77: 'Y', 81: 'Y', 97: 'C', 103: 'K', 110: 'G', 111: 'C', 114: 'R', 118: 'K', 119: 'A', 120: 'M', 121: 'N', 122: 'N', 127: 'N', 133: 'N', 136: 'N', 139: 'N'}, 'Bassia': {27: 'C', 35: 'T', 68: 'T', 70: 'A', 71: 'T', 77: 'T', 81: 'T', 97: 'C', 103: 'T', 110: 'G', 111: 'R', 114: 'A', 118: 'G', 119: 'R', 120: 'A', 121: 'W', 122: 'A', 127: '-', 133: '-', 136: '-', 139: '-'}, 'Suaeda': {27: 'T', 35: 'T', 68: 'G', 70: 'A', 71: 'T', 77: 'T', 81: 'T', 97: 'T', 103: 'T', 110: 'G', 111: 'C', 114: 'A', 118: 'G', 11

In [37]:
# %load -r 97-109 /Users/hughcross/scripts/synapomizer.py
# the informative_alleles function makes a list of informative alleles, then the next loop creates a filtered dictionary of dicts for each genus
filtered_chars_indiv = {}
for k,v in final_char_indiv.items():
    tax_dict = v  #final_chars[k]
    #print tax_dict
    for key, value in tax_dict.items():
        if key in inform_alleles:
            filtered_chars_indiv.setdefault(k, {})[key]=value

# now maybe sort the dictionary
#print(filtered_chars_indiv)
#print(name_map)
#now to deal with gaps?, maybe later

In [39]:
print(filtered_chars_indiv['Sclerolaena_cuneata_AD_b32'])

{27: 'C', 35: 'T', 68: 'T', 70: 'A', 71: 'T', 77: 'T', 81: 'C', 97: 'C', 103: 'T', 110: 'G', 111: 'G', 114: 'A', 118: 'G', 119: 'A', 120: 'A', 121: 'T', 122: 'A', 127: '-', 133: '-', 136: '-', 139: '-'}


In [32]:
# maybe make genotype dict for each genus (could also make it string)
genotypes = {} # dict of lists: {Felis: ['T', 'T',...]}
for k,v in filtered_chars1.items():
    for i in inform_alleles:
        geno = v[i]
        genotypes.setdefault(k, []).append(geno)
len(genotypes)

14

In [35]:
# maybe make genotype dict for each genus (could also make it string)
clade_genotypes = {} # dict of lists: {Felis: ['T', 'T',...]}
for k,v in filtered_chars_cld.items():
    for i in inform_alleles:
        geno = v[i]
        clade_genotypes.setdefault(k, []).append(geno)
len(clade_genotypes)
        

6

In [40]:
# maybe make genotype dict for each genus (could also make it string)
genotypes_indiv = {} # dict of lists: {Felis: ['T', 'T',...]}
for k,v in filtered_chars_indiv.items():
    for i in inform_alleles:
        geno = v[i]
        genotypes_indiv.setdefault(k, []).append(geno)
len(genotypes_indiv)
        

32

### Need to add CIGAR functions to read score from usearch output, and assign bp of each seq

In [43]:
clade_genotypes['saltbush']

['C',
 'Y',
 'T',
 'M',
 'N',
 'Y',
 'Y',
 'C',
 'K',
 'G',
 'C',
 'R',
 'K',
 'A',
 'M',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N']

## find differences between genotypes

In [ ]:
# start with clades
diffs = {}
for base in range(0,len(inform_alleles)):
    base_pos = inform_alleles[base]
    

In [42]:
inform_alleles

[27,
 35,
 68,
 70,
 71,
 77,
 81,
 97,
 103,
 110,
 111,
 114,
 118,
 119,
 120,
 121,
 122,
 127,
 133,
 136,
 139]

In [46]:
for key,value in clade_genotypes.items():
    print(key,'\t', value)

Salsola 	 ['C', 'T', 'T', 'C', 'T', 'A', 'C', 'C', 'T', 'G', 'G', 'A', 'G', 'A', 'A', 'G', 'A', '-', '-', '-', '-']
Tecticornia 	 ['C', 'T', 'T', 'A', 'T', 'A', 'C', 'C', 'T', 'T', 'A', 'A', 'G', 'G', 'A', 'A', '-', '-', '-', '-', '-']
saltbush 	 ['C', 'Y', 'T', 'M', 'N', 'Y', 'Y', 'C', 'K', 'G', 'C', 'R', 'K', 'A', 'M', 'N', 'N', 'N', 'N', 'N', 'N']
Bassia 	 ['C', 'T', 'T', 'A', 'T', 'T', 'T', 'C', 'T', 'G', 'R', 'A', 'G', 'R', 'A', 'W', 'A', '-', '-', '-', '-']
Suaeda 	 ['T', 'T', 'G', 'A', 'T', 'T', 'T', 'T', 'T', 'G', 'C', 'A', 'G', 'G', 'A', 'T', 'A', '-', '-', '-', '-']
bluebush 	 ['C', 'T', 'T', 'A', 'T', 'T', 'C', 'C', 'T', 'G', 'G', 'A', 'K', 'A', 'A', 'T', 'A', '-', '-', '-', '-']


In [48]:
print('saltbush\t',clade_genotypes['saltbush'])
print('bluebush\t',clade_genotypes['bluebush'])

saltbush	 ['C', 'Y', 'T', 'M', 'N', 'Y', 'Y', 'C', 'K', 'G', 'C', 'R', 'K', 'A', 'M', 'N', 'N', 'N', 'N', 'N', 'N']
bluebush	 ['C', 'T', 'T', 'A', 'T', 'T', 'C', 'C', 'T', 'G', 'G', 'A', 'K', 'A', 'A', 'T', 'A', '-', '-', '-', '-']
